In [1]:
import torch
import sys
sys.path.append('../')
import pickle
from models import dynamics_models as dm
from importlib import reload
import torch.autograd as autograd
from models import misc_mnet as mnet

reload(dm)

<module 'models.dynamics_models' from '../models/dynamics_models.py'>

In [2]:



inner_args = {'normalize': 0, 'batchsize': 32, 'num_loops': 20, 'detach': 1, 'beta': 0, 'temperature': 1, 'mode': 'exact', 'lr': 0.01, 'dim_a': 16}


In [3]:
with open('H0H1.pkl', 'rb') as fp:
    problem = pickle.load(fp)
H0 = problem['H0']
H1 = problem['H1']
import numpy as np


np.random.seed(1)
trueM = np.random.normal(size = (H0.shape[0], 16, 16))
trueH1 = H0 @ trueM
inner_args['num_loops']=200
inner_args['lr']=0.1  

In [4]:
dy_obj = dm.LinearTensorDynamicsLSTSQ_inner(inner_args=inner_args, verbose=True)

In [5]:
Mstar = dy_obj.compute_M(H0, trueH1)

inner_loss 3827.4614566244873
inner_loss 3732.5447568478608
inner_loss 3637.8722627095913
inner_loss 3543.4503328203828
inner_loss 3449.2856173397713
inner_loss 3355.3850748648606
inner_loss 3261.755989324226
inner_loss 3168.4059884303883
inner_loss 3075.3430841285904
inner_loss 2982.5756802561705
inner_loss 2890.1126159477885
inner_loss 2797.963188311904
inner_loss 2706.137194065424
inner_loss 2614.644964567518
inner_loss 2523.497409040959
inner_loss 2432.7060664575556
inner_loss 2342.2831532210066
inner_loss 2252.241624246099
inner_loss 2162.5952422839805
inner_loss 2073.358644398747
inner_loss 1984.547436134521
inner_loss 1896.1782768374917
inner_loss 1808.2689933522659
inner_loss 1720.8386996352783
inner_loss 1633.9079385112427
inner_loss 1547.4988437483992
inner_loss 1461.6353319933012
inner_loss 1376.343316191814
inner_loss 1291.650971284287
inner_loss 1207.5890415982997
inner_loss 1124.1912113707058
inner_loss 1041.4945538588997
inner_loss 959.5400763220724
inner_loss 878.373427

In [6]:
Mstar[0][:4, :4], trueM[0][:4, :4]

(tensor([[ 1.6243, -0.6118, -0.5282, -1.0730],
         [-0.1724, -0.8779,  0.0422,  0.5828],
         [-0.6872, -0.8452, -0.6712, -0.0127],
         [ 0.1202,  0.6172,  0.3002, -0.3522]], grad_fn=<SliceBackward>),
 array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862],
        [-0.17242821, -0.87785842,  0.04221375,  0.58281521],
        [-0.6871727 , -0.84520564, -0.67124613, -0.0126646 ],
        [ 0.12015895,  0.61720311,  0.30017032, -0.35224985]]))

In [7]:
torch.sqrt(torch.sum((H0@Mstar[:H0.shape[0]]- trueH1)**2))

tensor(2975.7557, dtype=torch.float64, grad_fn=<SqrtBackward>)

In [8]:
from torch.autograd import Variable


np.random.seed(1)


Meta_M = Variable(torch.tensor(torch.eye(inner_args['dim_a'])), requires_grad=True)
trueM_two = np.random.normal(size = (16, 16))

H0p = torch.tensor(np.random.normal(size=(H0.shape))).float()
H1hat = H0p @ trueM_two

inner_args['lr']=0.01
inner_args['num_loops']=20

/tmp/ipykernel_39/523019437.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Meta_M = Variable(torch.tensor(torch.eye(inner_args['dim_a'])), requires_grad=True)


In [9]:
with torch.enable_grad():

    for j in range(inner_args['num_loops']):
        loss = torch.sqrt(torch.sum((H0p@Meta_M - H1hat)**2))
        print(f"""inner_loss {loss.item()}""")
        grads = autograd.grad(loss, Meta_M,
                              create_graph=(not inner_args['detach']),
                              only_inputs=True, allow_unused=True)
        # parameter update
        # for param, grad in zip(self.mobject.parameters(), grads):
        #
        #     new_param = param - self.inner_args['lr'] * grad
        #     #new_param = param - 0.001 * grad
        #
        #     param.data.copy_(new_param)
        Meta_M = Meta_M - inner_args['lr'] * grads[0][0]
print("-"*10)
    


inner_loss 1440.6858096199555
inner_loss 1435.1323727087522
inner_loss 1435.4294134078739
inner_loss 1440.70853552326
inner_loss 1450.0891540032812
inner_loss 1462.7352938213508
inner_loss 1477.8927941927084
inner_loss 1494.907948424617
inner_loss 1513.2317452140246
inner_loss 1532.4145671460635
inner_loss 1552.095711255076
inner_loss 1571.9908055448618
inner_loss 1591.8791305376897
inner_loss 1611.5919792407444
inner_loss 1631.0024934877033
inner_loss 1650.0170887994427
inner_loss 1668.5684189456865
inner_loss 1686.609683032316
inner_loss 1704.110049850928
inner_loss 1721.051058728957
----------


In [10]:
Meta_M.shape

torch.Size([16, 16])

In [11]:
Meta_M.parameters

AttributeError: 'Tensor' object has no attribute 'parameters'

In [12]:
inner_args

{'normalize': 0,
 'batchsize': 32,
 'num_loops': 20,
 'detach': 1,
 'beta': 0,
 'temperature': 1,
 'mode': 'exact',
 'lr': 0.01,
 'dim_a': 16}

In [13]:
with open('H0H1.pkl', 'rb') as fp:
    problem = pickle.load(fp)
H0 = problem['H0']
H1 = problem['H1']
import numpy as np


np.random.seed(1)
trueM = np.random.normal(size = (H0.shape[0], 16, 16))
trueH1 = H0 @ trueM
inner_args['num_loops']=200
inner_args['lr']=0.01  



Meta_M_obj = mnet.Meta_Mnet(**inner_args)

In [14]:
with torch.enable_grad():

    for j in range(inner_args['num_loops']):
        loss =  Meta_M_obj(H0, H1hat)
        print(f"""inner_loss {loss.item()}""")
        grads = autograd.grad(loss, Meta_M_obj.parameters(),
                              create_graph=(not inner_args['detach']),
                              only_inputs=True, allow_unused=True)
        for param, grad in zip(Meta_M_obj.parameters(), grads):
        
            new_param = param - inner_args['lr'] * grad
        
            param.data.copy_(new_param)
        
print("-"*10)
    

inner_loss 1561.4898083037324


RuntimeError: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.

In [15]:
import copy
verbose=True
Mobj = mnet.Meta_Mnet(dim_a=16, batchsize=32, mode='exact')
#Mobj.Ms.data = copy.deepcopy(problem['M'])
#Mobj.Ms.data = torch.eye(16).float()
Mobj.Ms.data = torch.tensor(np.random.normal(size=(16, 16))).float()
with torch.enable_grad():

    for j in range(inner_args['num_loops']):
        loss = Mobj(H0, trueH1)
        grads = autograd.grad(loss, Mobj.parameters(),
                              create_graph=(not inner_args['detach']),
                              only_inputs=True, allow_unused=True)
        # parameter update
        for param, grad in zip(Mobj.parameters(), grads):

            #new_param = param - self.inner_args['lr'] * grad
            new_param = param - 0.1 * grad

            param.data.copy_(new_param)
        if verbose: print(f"""inner_loss {loss.item()}""")
    if verbose: print("-"*10)
Mstar = Mobj.Ms.data

inner_loss 3889.787199871139
inner_loss 2792.8971687002277
inner_loss 2684.051761452392
inner_loss 2678.1238944291563
inner_loss 2677.6739869674684
inner_loss 2677.629693329063
inner_loss 2677.623908730731
inner_loss 2677.6228541944038
inner_loss 2677.6226021516727
inner_loss 2677.62253230466
inner_loss 2677.622512086098
inner_loss 2677.6225056735566
inner_loss 2677.622503809284
inner_loss 2677.622503347045
inner_loss 2677.6225033246465
inner_loss 2677.622503158332
inner_loss 2677.622502989294
inner_loss 2677.622503079189
inner_loss 2677.6225031763
inner_loss 2677.622503056528
inner_loss 2677.622503180004
inner_loss 2677.6225030693067
inner_loss 2677.6225029835796
inner_loss 2677.6225031918543
inner_loss 2677.62250289674
inner_loss 2677.622502863409
inner_loss 2677.6225031131503
inner_loss 2677.622503137011
inner_loss 2677.6225032623415
inner_loss 2677.6225031148124
inner_loss 2677.6225033618935
inner_loss 2677.6225031510357
inner_loss 2677.6225030674627
inner_loss 2677.6225033608007
i

In [16]:
reload(mnet)
Mobj = mnet.Meta_Mnet(dim_a=16, batchsize=32, mode='exact')
optimizer = torch.optim.SGD(Mobj.parameters(), inner_args['lr'])

for k in range(inner_args['num_loops']):
    loss = Mobj(H0, trueH1)
    print(f"""inner_loss {loss.item()}""")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


inner_loss 2801.8849043935033


RuntimeError: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.

In [17]:
H0

tensor([[[-2.1109, -2.4911,  1.7026,  ..., -0.4438, -1.3853, -1.1067],
         [-1.7529,  2.8129, -2.4634,  ...,  2.4947, -1.7529, -1.5583],
         [-2.0048, -2.1789, -2.0868,  ..., -2.2929,  1.9419, -1.1916],
         ...,
         [ 1.0895, -1.4626,  2.6042,  ...,  1.7241,  0.4403, -0.4087],
         [ 1.8314, -2.6205, -2.5710,  ...,  0.9191, -2.5827, -1.9482],
         [-1.5201,  1.8679, -2.0569,  ...,  2.2132,  2.3692,  2.5352]],

        [[-1.8675, -2.2506,  1.3423,  ..., -0.1323, -0.9872, -0.6825],
         [-2.0292,  2.6892, -2.0201,  ...,  1.9581, -1.6889, -1.2036],
         [-2.0917, -2.0253, -2.2447,  ..., -2.4576,  2.4662, -1.4204],
         ...,
         [ 1.5217, -1.2507,  2.4432,  ...,  1.4848,  0.5541, -0.4833],
         [ 1.9434, -2.5379, -2.5964,  ...,  0.9308, -2.5965, -1.9227],
         [-1.7474,  1.9326, -2.1185,  ...,  2.1439,  2.3942,  2.2156]],

        [[-1.8111, -2.4227,  1.1315,  ...,  0.6633, -0.9333, -0.6034],
         [-2.2678,  2.4463, -2.2146,  ...,  2

In [18]:
H1

tensor([[[-2.0258, -2.1756,  1.6909,  ..., -0.4623, -1.2928, -1.0002],
         [-1.8270,  2.9528, -2.3075,  ...,  2.7862, -1.7894, -1.6834],
         [-2.0330, -2.0211, -2.3528,  ..., -2.1145,  1.8305, -0.8984],
         ...,
         [ 1.3440, -1.3396,  2.2562,  ...,  1.5883,  0.3422, -0.3967],
         [ 1.8721, -2.6913, -2.2773,  ...,  1.0807, -2.2359, -1.8012],
         [-1.6778,  1.9449, -2.0263,  ...,  2.0222,  2.2672,  2.5387]],

        [[-2.0626, -2.1800,  1.4792,  ...,  0.2510, -1.0677, -0.7235],
         [-2.0879,  2.7920, -2.2749,  ...,  2.1215, -1.8976, -1.5059],
         [-1.9622, -1.9252, -2.0342,  ..., -2.6619,  2.3244, -1.5062],
         ...,
         [ 1.3953, -1.3354,  2.5230,  ...,  1.8409,  0.5737, -0.6423],
         [ 2.1262, -2.6986, -2.3735,  ...,  1.1961, -2.3959, -1.9148],
         [-1.5842,  2.2761, -2.1835,  ...,  2.0623,  2.0234,  2.4233]],

        [[-1.4912, -2.2418,  1.3095,  ...,  0.4815, -1.1305, -0.3138],
         [-2.0136,  2.3749, -2.2645,  ...,  2